# Implementation of Trie 

In [85]:
class Node:
    def __init__(self, depth, parent, char):
        self.depth    = depth
        self.parent   = parent
        self.v        = char
        self.children = []

    ## Looks for children with v = char.
    def find_child(self, char):
        for child in self.children:
            if child.v == char:
                return child
        
        return None

    def search(self, pattern):
        if len(pattern)  <= 0:
            return True

        child = self.find_child(pattern[0])
        if child == None:
            return False
        else:
            return child.search(pattern[1:])

    def graft(self, suffix_end):
        parent        = self
        running_depth = self.depth + 1

        for char in suffix_end:
            child = Node(running_depth, parent, char)
            parent.children.append(child)
            running_depth += 1
            parent = child
        
        return parent

In [86]:
class Trie:
    def __init__(self, marker = None):
        self.root = Node(0, None, None)    
        self.marker = marker

    def search_pattern(self, pattern):
        return self.root.search(pattern)

    def insert(self, text):
        if self.marker:
            text += self.marker

        for i in range(len(text)):
            suffix = text[i:]
            head   = self.find(suffix)
            suffix_end = suffix[head.depth:]
            head.graft(suffix_end)

    def find(self, suffix):
        head = self.root

        for char in suffix:
            child = head.find_child(char)
            if child == None:
                break
            else:
                head = child
        
        return head

In [79]:
def ensure_marker(text):
    last = text[len(text) - 1]
    if last not in text[:len(text) - 1]:
        return None
    
    i = 0
    while True:
        marker = chr(ord('$') + i)
        if marker not in text:
            return marker
        i += 1

def build_tree_schema(text, insert = True):
    marker = ensure_marker(text)
    trie = Trie(marker)
    if insert:
        trie.insert(text)

    return trie

In [80]:
text = "aabbbd"
trie = build_tree_schema(text)
trie.search_pattern("bd")

True

## Testing trie

I test implemented trie by checking if i can
find all subwords of inserted words. 

Building trie for text from file took too long so instead I broke it down to single words and inserted them.
For some subset of this words i generated all subwords and checked if they're in the structure.

In [87]:
# Todo replace time with timeit!!!
import time

simple_tests = ['bbb$', 'aabbabd', 'ababcd', 'abcbccd']

def substring_generator(word):
    n = len(word)
    for i in range(n - 1):
        for j in range(i + 1, n):
            yield word[i:j]

## Tests by generating all subwords and checking if it can be found in generated trie.
def test_simple_trie(word):
    generator = substring_generator(word)
    start     = time.time()
    trie      = build_tree_schema(word)
    for subword in generator:
        if not trie.search_pattern(subword):
            print( u'\u2718', "'{}'".format(word), 'failed.')
            print(subword, 'not found in trie.')
            return
    end = time.time()
    print(u'\u2713', "'{}'".format(word), f'passed with time {end - start}s')


## Building trie for whole text takes too long so 
## to measure it somehow i insert invidual words which
## enables me to search for particular words and subwords.
def test_file_trie(filename = '1997_714.txt'):
    trie = None
    start = time.time()
    with open(filename, 'r', encoding = 'UTF-8') as file:
        content = file.read()
        trie     = build_tree_schema(content, insert = False)

    selected = []
    with open(filename, 'r', encoding = 'UTF-8') as file:  
        i = 0;
        for line in file.readlines():
            for word in line.split():
                trie.insert(word)
                i += 1
                if i % 100 == 0:
                    selected.append(word)

    for word in selected:
        generator = substring_generator(word)
        for subword in generator:
            if not trie.search_pattern(subword):
                print( u'\u2718', "'{}'".format(word), 'failed.')
                print(subword, 'not found in trie.')
                return
    end = time.time()
    print(u'\u2713', "'{}'".format(filename), f'passed with time {end - start}s')
    print(f'\tTested all subwords of selected {i // 100} words.')


def test_suite_trie():
    for word in simple_tests:
        test_simple_trie(word)
    
    test_file_trie()

In [88]:
test_suite_trie()

✓ 'bbb$' passed with time 0.0s
✓ 'aabbabd' passed with time 0.0s
✓ 'ababcd' passed with time 0.0s
✓ 'abcbccd' passed with time 0.0s
✓ '1997_714.txt' passed with time 1.7960624694824219s
	Tested all subwords of selected 259 words.


# Implementation of suffix tree without using links and fast find

In [95]:
class SuffixTreeNode:
    def __init__(self, depth, parent, bounds):
        self.depth    = depth
        self.parent   = parent
        self.bounds   = bounds
        self.children = []

    def label_length():
        return self.bounds[1] - self.bounds[0]

    ## Returns tuple (child, matching characters of this child)
    def find_child(self, text, str):
        for child in self.children:
            if text[child.bounds[0]] == str[0]:
                matching = 1
                for i in range(1, min(len(str), child.bounds[1])):
                    if text[child.bounds[0] + i] == str[i]:
                        matching += 1
                    else:
                        break

                return (child, matching)
        
        return (None, None)

    ## Where cut tells us how many characters are matching
    def breakMe(self, cut):
        child_bounds_start = self.bounds[0] + cut
        child_bounds_end   = self.bounds[1]
        child_depth        = self.depth + cut 

        child = SuffixTreeNode(child_depth, self, (child_bounds_start, child_bounds_end))
        child.children = self.children
        
        self.bounds[1] = new_bounds_start
        self.children  = []

        return self

    def graft(self, bounds):
        parent = self
        depth  = self.depth + (self.bounds[1] - self.bounds[0]) 

        node = SuffixTreeNode(depth, self, bounds)
        self.children.append(node)
        
        return node

    def search(self, text, pattern):
        if len(pattern) <= 0:
            return True
        child, matching = self.find_child(text, pattern)
        
        if child == None:
            return False
        else:
            return child.search(text, pattern[matching:])

In [96]:
class SuffixTree:
    def __init__(self, text):
        self.root = SuffixTreeNode(0, None, (0, 0))
        self.text = ensure_marker(text) 

    def search_pattern(self, pattern):
        return self.root.search(self.text, pattern)

    def find(self, suffix):
        head = self.root
        child = head

        while True:
            head  = child
            (child, matching) = child.find_child(self.text, suffix)

            if not child:
                break
            if child.label_length != matching:
                break
        
        return head

def ensure_marker(text):
    last = text[len(text) - 1]
    if last not in text[:len(text) - 1]:
        return text
    
    i = 0
    while True:
        marker = chr(ord('$') + i)
        if marker not in text:
            return text + marker
        i += 1

def build_tree_schema(text, insert = True):
    suffix_tree = SuffixTree(text)
    suffix_tree.root.graft((0, len(text)))

    for i in range(1, len(text)):
        suffix = text[i:]
        head   = suffix_tree.find(suffix)
        head.graft((head.depth, len(text)))

    return suffix_tree